In [200]:
import numpy as np
import matplotlib.pyplot as plt
import random as rdm
from IPython import display
#from matplotlib import animation
from matplotlib.pyplot import Circle
from matplotlib.animation import FuncAnimation
import warnings

In [201]:
class make_walker:
    def __init__(self,x,y,m,v,r,i,color):
        self.x = x
        self.y = y
        self.v = v
        self.m = m
        self.r = r
        self.i = i
        self.color = color

In [209]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Initialize walkers in space
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(1,2,1)
ax1.axis('scaled')
ax2 = fig.add_subplot(1,2,2)
ax2.axis('auto')

n = 10 # number of walkers
x_min, x_max = 0, 100
y_min, y_max = 0, 100
m_min, m_max = 1, 1
v_min, v_max = 1, 1
r_min, r_max = 3, 3
dt = 1 # timescale
Nframes = 200
x_data = []
y_data = []

ax1.set_xticks([x_min,x_max])
ax1.set_yticks([y_min, y_max])

walkers_list = []
temp = [[0,0,0]]

for i in range(n):
    m = m_min + rdm.random()*(m_max-m_min)
    r = round(r_min + (r_max-r_min)*rdm.random())
    x = x_min + r + rdm.random()*(x_max-x_min-2*r)
    y = y_min + r + rdm.random()*(y_max-y_min-2*r)
    restart = True
    counter = 0
    while restart:
        restart = False
        for j in temp:
            while np.sqrt((x-j[0])**2 + (y-j[1])**2) <= (r+j[2]):
                restart = True
                x = x_min + r + rdm.random()*(x_max-x_min-2*r)
                y = y_min + r + rdm.random()*(y_max-y_min-2*r)
                counter = counter + 1
                if counter >= 1e6:
                    raise ValueError('too many attempts to place walker')
                break
    temp.append([x,y,r])
    v = [v_min + rdm.random()*(v_max-v_min), v_min + rdm.random()*(v_max-v_min)]
    v = [a*b for a,b in zip(v,rdm.choices([-1,1],k=2))]
    i = 0
    color = 'blue'
    walkers_list.append(make_walker(x,y,m,v,r,i,color))
del temp

patient_0 = walkers_list[rdm.randint(0,n)]
patient_0.i, patient_0.color = 1, 'red'
    
patches_list = []
for w in walkers_list:
    patches_list.append(ax1.add_patch(Circle((w.x, w.y), radius=w.r, edgecolor='blue', facecolor=w.color, alpha=0.5)))

    
def animate(frame):

    #for w in walkers_list:
    for c in range(0,len(walkers_list)):
        
        w = walkers_list[c]
        # update the position using velocity
        w.x = w.x + w.v[0]*dt
        w.y = w.y + w.v[1]*dt

        # check if there are overlaps
        for q in walkers_list:
            
            # check each particle for a colision
            if np.sqrt((w.x-q.x)**2 + (w.y-q.y)**2) <= (w.r+q.r) and q != w:
                
                if q.i == 1:
                    w.i, w.color = q.i, q.color
                if w.i == 1:
                    q.i, q.color = w.i, w.color
                
                # 2d momentum problem
                v1, v2 = np.array(w.v), np.array(q.v)
                m1, m2 = w.m, q.m
                d1, d2 = np.array([w.x, w.y]), np.array([q.x, q.y])
                
                w.v = v1 - (2*m2)/(m1+m2) * (d1-d2) * np.dot(v1-v2,d1-d2) / np.linalg.norm(d1 - d2)**2
                q.v = v2 - (2*m1)/(m1+m2) * (d2-d1) * np.dot(v2-v1,d2-d1) / np.linalg.norm(d2 - d1)**2

                while np.sqrt((w.x-q.x)**2 + (w.y-q.y)**2) <= (w.r+q.r):
                    w.x = w.x + w.v[0]*dt
                    w.y = w.y + w.v[1]*dt
                    q.x = q.x + q.v[0]*dt
                    q.y = q.y + q.v[1]*dt 
                    
        if w.x-w.r <= x_min:
            w.v[0] = -1*w.v[0]
            while w.x-w.r <= x_min:
                w.x = w.x + w.v[0]*dt
        if w.x+w.r >= x_max:
            w.v[0] = -1*w.v[0]
            while w.x+w.r >= x_max:
                w.x = w.x + w.v[0]*dt
        if w.y-w.r <= y_min:
            w.v[1] = -1*w.v[1]
            while w.y-w.r <= y_min:
                w.y = w.y + w.v[1]*dt
        if w.y+w.r >= y_max:
            w.v[1] = -1*w.v[1]
            while w.y+w.r >= y_max:
                w.y = w.y + w.v[1]*dt


        patches_list[c].set_center([w.x, w.y])
        patches_list[c].set_facecolor(w.color)
        
    num_infected = 0
    for patient in walkers_list:
        if patient.i == 1:
            num_infected = num_infected + 1
        
    x_data.append(frame)
    y_data.append(num_infected)
    ax2.plot(x_data,y_data, color = 'blue', linewidth=.5)

    return patches_list # The animation function must return a sequence of Artist objects.


anim = animation.FuncAnimation(fig, animate, frames=Nframes, interval=30, blit=True)
video = anim.to_html5_video()
plt.close()
display.HTML(video)
    

